# AutoGPT

From https://github.com/langchain-ai/langchain/blob/master/cookbook/autogpt/autogpt.ipynb

Implementation of https://github.com/Significant-Gravitas/Auto-GPT but with LangChain primitives (LLMs, PromptTemplates, VectorStores, Embeddings, Tools)

In [6]:
from langchain.agents import Tool
from langchain_community.tools.file_management.read import ReadFileTool
from langchain_community.tools.file_management.write import WriteFileTool
from langchain.tools import ElevenLabsText2SpeechTool
from langchain_community.utilities import GoogleSerperAPIWrapper
from dotenv import load_dotenv

from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

from langchain_experimental.autonomous_agents import AutoGPT
from langchain_openai import ChatOpenAI
from langchain_community.llms import Ollama

import faiss

## Set up memory
The memory here is used for the agents intermediate steps

In [7]:
load_dotenv('.env')

search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name="search",
        func=search.run,
        description="Useful for when you need to answer questions about current events. You should ask targeted questions",
    ),
    WriteFileTool(),
    ReadFileTool(),
    ElevenLabsText2SpeechTool()
]

In [8]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


## Setup model and AutoGPT

Initialize everything! We will use ChatOpenAI model

In [9]:
agent = AutoGPT.from_llm_and_tools(
    ai_name="Tom",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(temperature=0.5),
    # llm = Ollama(temperature=0.5, model="mistral"),
    memory=vectorstore.as_retriever(),
)
# Set verbose to be true
agent.chain.verbose = True

## Run an example

Here we will make it write a weather report for SF

In [10]:
agent.run([
    "Tell me a weather report for Santa Barbara on Saturday, in the style of a British news presenter that says even the most mundane things in a deep and profound way. You MUST be deep and profound, or else you will not be paid. Make it a full-length weather report that would take a presenter about 30 seconds to deliver. Give as many statistical figures as are relevant. Write your speech to an audio file.",
])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are Tom, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. Tell me a weather report for Santa Barbara on Saturday. You are a British news presenter that says even the most mundane things in a deep and profound way. You MUST be deep and profound, or else you will not be paid. Make it a full-length weather report that would take a presenter about 30 seconds to deliver. Give as many statistical figures as are relevant. Write your speech to an audio file, and use the voice style that is consistent with your character.


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did

KeyboardInterrupt: 

## Chat History Memory

In addition to the memory that holds the agent immediate steps, we also have a chat history memory. By default, the agent will use 'ChatMessageHistory' and it can be changed. This is useful when you want to use a different type of memory for example 'FileChatHistoryMemory'

In [ ]:
from langchain_community.chat_message_histories import FileChatMessageHistory

agent = AutoGPT.from_llm_and_tools(
    ai_name="Tom",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(temperature=0),
    memory=vectorstore.as_retriever(),
    chat_history_memory=FileChatMessageHistory("chat_history.txt"),
)